# Shape descriptors  
##### Characterizing the shape of a segmented image (binary mask), that can be either the entire Placozoa organism, either the wound.

Import all the mandatory modules

In [2]:
import skimage
import napari
from tifffile import imread
from pathlib import Path
import scipy.ndimage as ndi

import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from skimage.draw import ellipse
from skimage.measure import label, regionprops, regionprops_table,perimeter
from skimage.transform import rotate
from skimage.morphology import convex_hull_image

Indicate the original image name, the mask name (tif files that are normally in the "data" folder).
Indicate the name of the csv file, that will correspond to the final table with the analyzed data.

In [4]:
# You might need to modify the path according to where you saved your dataset
# Note that the `Path` library will deal with OS dependant path issues
path_img = Path('../data/placozoan-movie.tif')#path of the original image
path_label = Path('../data/final_mask_wound.tif')#path of the mask: the filename of the wound mask and the filename of the organism mask are two different files. Don't forget to change it: 'labeled_wound.tif','final_mask_wound.tif','imseq_mask_filtered.tif'
path_output_table = Path('../data/ppties_wound.csv')#path to save data as csv file. Don't forget to change it if you run the shape descriptor analysis for the organism or for the wound, for the same sample;
#names : ppties.csv or ppties_wound.csv

#read the files
img = imread(path_img)
labels = imread(path_label)


In [350]:
# temporary, if you want to test on a small substet of data
#img = img[:700]
#labels = labels[:700].astype(np.int8)

In [5]:
#fill the holes within the mask
label_filled=ndi.binary_fill_holes(labels)
# it will be useful to compute the right object perimeter when using regionprops

Choose the properties you would like to extract from the binary mask (= segmentation):  
https://scikit-image.org/docs/dev/api/skimage.measure.html#skimage.measure.regionprops_table  
Add these components to the 'properties' list, below.


In [6]:
properties=['label','centroid','area', 'perimeter','eccentricity','orientation','axis_major_length','axis_minor_length']

In [7]:
## Initialize the dataframe (final table) with timepoint 0. 
tab=pd.DataFrame(regionprops_table(   
        label_filled[0].astype(int),
        intensity_image=img[0],
        properties=properties,
    ))

In [8]:
## Then we will compute the metrics for each timepoint
## and concatenate the new line (corresponding to the new timepoint) to the dataframe
for t in range (1,len(labels)) :
    other_timepoints=pd.DataFrame(regionprops_table(   
        (label_filled[t]*(t+1)).astype(int),
        intensity_image=img[t],
        properties=properties,
    ))
    tab=pd.concat([tab, other_timepoints])

We then compute the convexity, that was not available from the regionprops modules. We then add it to the dataframe.  
 
The convexity is the ratio between the perimeter of the convex hull of an object and the real perimeter of the object. If it is lower than 1, it indicates that the object is highly concave at some part of its shape.

In [9]:
#compute convexity
#actually you can add whatever metric you want to the dataframe
list_convexity=[]
for t in range (0,len(labels)) : #for each timepoint
    #print(t)
    conv_hull=convex_hull_image(label_filled[t]) #compute the convex hull of the current image mask
    perimeter_convhull=perimeter(conv_hull) #compute its perimeter
    perimeter_mask=perimeter(label_filled[t]) #compute the perimeter of the current image mask

    convexity=perimeter_convhull/perimeter_mask #compute their ratio
    
    if math.isnan(convexity) == 0:
        list_convexity.append(convexity) #we keep the data only when the mask exists

tab['convexity']=list_convexity

C:\Users\gros\AppData\Local\Temp\ipykernel_8324\3564871535.py:6: UserWarning: Input image is entirely zero, no valid convex hull. Returning empty image
  conv_hull=convex_hull_image(label_filled[t]) #compute the convex hull of the current image mask
C:\Users\gros\AppData\Local\Temp\ipykernel_8324\3564871535.py:10: RuntimeWarning: invalid value encountered in double_scalars
  convexity=perimeter_convhull/perimeter_mask #compute their ratio


In [10]:
## sort the dataframe by label index
ppties=tab.set_index('label')

Save the dataframe as a csv file :

In [11]:
ppties.to_csv(path_output_table,sep=';')